# IHSG Pre-processing Data

### Audhi Aprilliant

## 1 Import Libraries

In [89]:
import pandas as pd                   # Dataframe manipulation
import numpy as np                    # Mathematics operation
import datetime

## Load the Data

In [90]:
ihsg_data = pd.read_csv('Datasets/^JKSE.csv')

In [91]:
print('Dimension of financial news:\n{}'.format(ihsg_data.shape[0]),
      'rows and {}'.format(ihsg_data.shape[1]),'columns')
ihsg_data.head(6)

Dimension of financial news:
257 rows and 7 columns


,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-03-08,6439.840820,6442.204102,6381.730957,6383.067871,6383.067871,92588600.0
1,2019-03-11,6418.882813,6420.782227,6366.039063,6366.434082,6366.434082,97316700.0
2,2019-03-12,6395.880859,6404.430176,6339.967773,6353.773926,6353.773926,84514000.0
3,2019-03-13,6343.121094,6377.575195,6337.874023,6377.575195,6377.575195,100029400.0
4,2019-03-14,6374.916992,6413.266113,6372.964844,6413.266113,6413.266113,96692700.0
5,2019-03-15,6420.182129,6489.194824,6419.644043,6461.183105,6461.183105,122938800.0


In [92]:
ihsg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 7 columns):
Date         257 non-null object
Open         247 non-null float64
High         247 non-null float64
Low          247 non-null float64
Close        247 non-null float64
Adj Close    247 non-null float64
Volume       247 non-null float64
dtypes: float64(6), object(1)
memory usage: 14.2+ KB


In [93]:
# Check missing value
ihsg_data.isna().sum()

Date          0
Open         10
High         10
Low          10
Close        10
Adj Close    10
Volume       10
dtype: int64

## 2 Function to Imput the Date and Time

In [94]:
def imput_date(df):
    start = datetime.datetime.strptime(df.loc[0,'Date'],'%Y-%m-%d')
    end = datetime.datetime.strptime(df.loc[df.shape[0] - 1,'Date'],'%Y-%m-%d')
    list_date = pd.date_range(start,end).strftime('%Y-%m-%d').tolist()
    pd_date = pd.DataFrame(df,list_date)
    pd_date['Date'] = pd_date.index.astype(object)
    pd_date = pd_date.reset_index(drop=True)
    df_date = pd.merge(pd_date['Date'],df,on='Date',how='left')
    return df_date

## 2 Function to Calculate the Return of IHSG

In [95]:
def return_ihsg(df):
    df_no_na = df.dropna().reset_index(drop=True)
    return_data = [0]
    index_data = list(df_no_na.index)
    for i in range(1,len(index_data)):
        return_i = (df_no_na['Adj Close'][index_data[i]] - 
                    df_no_na['Adj Close'][index_data[i-1]])/df_no_na['Adj Close'][index_data[i]]
        return_data.append(return_i)
    df_no_na = pd.concat([df_no_na['Date'],pd.DataFrame(return_data)],axis=1)
    df_full = pd.merge(df,df_no_na,on='Date',how='left')
    df_full = df_full.rename(columns = {0:'Return'})
    return df_full

## 3 Function to Apply Curve Function

In [96]:
def curve_function(df):
    for i in df.columns:
        while df[i].isna().sum() > 0:
            for j in range(df.shape[0]):
                if pd.isnull(df.loc[j,i]):
                    seq_k = [j]
                    k = j
                    while pd.isnull(df.loc[k,i]):
                        k = k + 1
                        seq_k.append(k)
                    if len(seq_k) % 2 == 0:
                        df.loc[seq_k[int((len(seq_k) - 1)/2)],i] = (df.loc[j - 1,i] + 
                                                                    df.loc[seq_k[len(seq_k) - 1],i])/2
                    else:
                        df.loc[seq_k[int((len(seq_k) - 1)/2)],i] = (df.loc[j - 1,i] + 
                                                                    df.loc[seq_k[len(seq_k) - 1],i])/2
                else:
                    df.loc[j,i] = df.loc[j,i]
    return(df)

## 4 Apply Functions Above

In [97]:
ihsg_data_clean = return_ihsg(ihsg_data)
print('Dimension of financial news:\n{}'.format(ihsg_data_clean.shape[0]),
      'rows and {}'.format(ihsg_data_clean.shape[1]),'columns')
ihsg_data_clean.head()

Dimension of financial news:
257 rows and 8 columns


,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,2019-03-08,6439.840820,6442.204102,6381.730957,6383.067871,6383.067871,92588600.0,0.000000
1,2019-03-11,6418.882813,6420.782227,6366.039063,6366.434082,6366.434082,97316700.0,-0.002613
2,2019-03-12,6395.880859,6404.430176,6339.967773,6353.773926,6353.773926,84514000.0,-0.001993
3,2019-03-13,6343.121094,6377.575195,6337.874023,6377.575195,6377.575195,100029400.0,0.003732
4,2019-03-14,6374.916992,6413.266113,6372.964844,6413.266113,6413.266113,96692700.0,0.005565


In [98]:
ihsg_data_clean = imput_date(ihsg_data_clean)
print('Dimension of financial news:\n{}'.format(ihsg_data_clean.shape[0]),
      'rows and {}'.format(ihsg_data_clean.shape[1]),'columns')
ihsg_data_clean.head()

Dimension of financial news:
365 rows and 8 columns


,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,2019-03-08,6439.840820,6442.204102,6381.730957,6383.067871,6383.067871,92588600.0,0.000000
1,2019-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-03-11,6418.882813,6420.782227,6366.039063,6366.434082,6366.434082,97316700.0,-0.002613
4,2019-03-12,6395.880859,6404.430176,6339.967773,6353.773926,6353.773926,84514000.0,-0.001993


In [99]:
ihsg_data_curve = curve_function(ihsg_data_clean)
print('Dimension of financial news:\n{}'.format(ihsg_data_curve.shape[0]),
      'rows and {}'.format(ihsg_data_curve.shape[1]),'columns')
ihsg_data_curve.head()

Dimension of financial news:
365 rows and 8 columns


,Date,Open,High,Low,Close,Adj Close,Volume,Return
0,2019-03-08,6439.840820,6442.204102,6381.730957,6383.067871,6383.067871,92588600.0,0.000000
1,2019-03-09,6434.601318,6436.848633,6377.807983,6378.909424,6378.909424,93770625.0,-0.000653
2,2019-03-10,6429.361817,6431.493165,6373.885010,6374.750976,6374.750976,94952650.0,-0.001306
3,2019-03-11,6418.882813,6420.782227,6366.039063,6366.434082,6366.434082,97316700.0,-0.002613
4,2019-03-12,6395.880859,6404.430176,6339.967773,6353.773926,6353.773926,84514000.0,-0.001993


## Save Data

In [100]:
ihsg_data_curve.to_csv("Datasets/interim/Clean Data of JCI 2019.csv",index=False)